In [9]:
import pandas as pd
import spacy

from tqdm.notebook import tqdm
tqdm.pandas()



Ce code réalise une lemmatization à partir d'un modèle de machine learning issu de spacy.
installation du modèle : 
> python -m spacy download fr_core_news_sm

Ensuite on supprime certains mots fréquents, remis en forme et trouvés grâce à :
> from nltk.probability import FreqDist

Les nombres sont aussi filtrés.

J'ai réalisé cette approche car un  cleaning manuel était trop long (code en n² où n est le nombre de mots total du corpus). Le défaut de cette aproche est que potentiellement est elle trop bourrine et certains mots spécifiques qui nous intéressent ont disparu.

Il faut valider par une approche lexico et LDA les résultats => cf le code analyse_lexico.qmd

# Opening data

In [10]:
fichier = "../data/intermediate/base_merged.csv"

df = pd.read_csv(fichier, sep=';', quotechar='"')

df.sample(5)

,ID ASSO,ID ARTICLE,contenu,Type de document,Auteur,Date,Titre,URL
460,13,130090,Reportage du journal The Telegraph sur les éch...,blog,TRADFEM,29/05/2023,Reportage du journal The Telegraph sur les éch...,https://tradfem.wordpress.com/2023/05/29/repor...
204,19,190002,Les rapports contradictoires du féminisme aux ...,Article d'opinion,NaN,NaN,NaN,https://blogs.mediapart.fr/nicole-roelens-deke...
69,17,170041,"En ce vendredi 29 novembre 2013, j’ai passé l’...",analyse,NaN,NaN,NaN,NaN
40,17,170012,"Christine Delphy, sociologue française, a fait...",analyse,NaN,NaN,NaN,NaN
407,13,130037,Le porno comme source occulte de l’idéologie t...,blog,TRADFEM,17/08/2023,Le porno comme source occulte de l’idéologie t...,https://tradfem.wordpress.com/2023/08/17/le-po...


# Cleaning and tokenisation

In [11]:
# Load the French language model
nlp = spacy.load("fr_core_news_sm")

# Define your list of words to exclude
words_to_filter = [
    'le', 'la', 'les', 'de', 'un', 'à', 'être', 'suis', 'es', 'est', 'sont', 'été', 'étais', 'était', 'étaient', 'serai', 'seras', 'sera', 'seront',
    'et', 'en', 'avoir', 'ai', 'as', 'a', 'ont', 'eu', 'aurai', 'auras', 'aura', 'auront', 'avais', 'avait', 'avaient', 'que', 'pour',
    'dans', 'ce', 'il', 'qui', 'ne', 'en', 'sur', 'se', 'pas', 'plus',
    'par', 'je', 'avec', 'tout', 'faire', 'fais', 'fait', 'font', 'son', 'mettre', 'autre', 'on', 'mais',
    'nous', 'comme', 'ou', 'si', 'leur', 'y', 'dire', 'dis', 'dit', 'disent', 'elle', 'voir', 'devoir',
    'deux', 'même', 'pendant', 'aussi', 'vouloir', 'grand', 'mon', 'nouveau', 'aller',
    'venir', 'ceux', 'faire', 'quelque', 'trouver', 'donner', 'donne', 'donnes', 'donnent', 'aussi', 'autre',
    'tous', 'vieux', 'bon', 'voir', 'moins', 'trois', 'avant', 'sa', 'faire', 'contre',
    'abord', 'sous', 'ou', 'apporter', 'grand', 'ainsi', 'long', 'très', 'tout', 'avoir',
    'beau', 'chaque', 'peu', 'quoi', 'encore', 'aller', 'montrer', 'semaine', 'ainsi',
    'nuit', 'aussi', 'bien', 'deuxième', 'moins', 'tout', 'avoir', 'peu', 'nuit', 'ça',
    'mon', 'ma', 'mes', 'ton', 'ta', 'tes', 'son', 'sa', 'ses', 'notre', 'notre', 'nos', 'votre', 'votre', 'vos', 'leur', 'leur', 'leurs',
    'non', '’', '.', ',', ':', 'est', 'des', "l'", "d'", "une", "d", "»", "«", "du", "qu", "au", "n", "s", "elle", ")", "(", "aux", 'c', 'cette', 'un', 'ces','?', 'il', 'ils', 'j', "l", "-", "vous", "entre",
    "[", "]", "elles", '”', '“', "dont", "cela", ";", "m", "selon", "!", '–', "en", "autre", "ici", "ce", "ça", "cela", "ceci", "car", "ou", "où", "ni", "mais", "et", "donc", "parce", "que", "quand", "comment", "qui", "quoi",
    "à", "de", "pour", "sans", "sur"
]

# Function to tokenize, filter, and lemmatize text, excluding numbers
def tokenize_filter_and_lemmatize(text):
    # Process the text using spaCy
    doc = nlp(text)
    # Extract lemmatized forms of the words, excluding specific tokens and numbers
    lemmatized = " ".join([token.lemma_ for token in doc if token.lemma_ not in words_to_filter and not token.is_digit])
    return lemmatized

# Apply the function to the 'contenu' column
df['lemmatized_contenu'] = df['contenu'].progress_apply(tokenize_filter_and_lemmatize)

# Print the dataframe to see the result
df.sample(5)

  0%|          | 0/488 [00:00<?, ?it/s]

,ID ASSO,ID ARTICLE,contenu,Type de document,Auteur,Date,Titre,URL,lemmatized_contenu
53,17,170025,"11 août 2014 actualisation\n Dans ce texte, j...",analyse,NaN,NaN,NaN,NaN,août actualisation \n texte n’ jamais préte...
111,17,170083,"Cher C. M. Si vous ne le savez pas, je vous in...",analyse,NaN,NaN,NaN,NaN,cher c. m. savez informe lettre adresser prési...
213,11,110001,la question tr ansg enrechez les enfantscessez...,Note de synthèse,NaN,NaN,NaN,https://soseducation.org/docs/notes-etudes-ent...,question tr ansg enrecher enfantscessez dégenr...
251,20,200023,"Dans plusieurs endroits du monde, dont l’Europ...",Article d'opinion,NaN,NaN,NaN,https://cryforrecognition.be/fr/protegeons-les...,plusieurs endroit monde l’ Europe nombreux cit...
334,30,300015,"S’affirmant garçon et non pas fille, une enfan...",Article d'opinion,NaN,2014-11-04 00:00:00,NaN,https://www.parti-de-la-france.fr/articles/Pro...,s’ affirmer garçon fille enfant an choisir sou...


# Export

In [12]:
fichier = "../data/intermediate/base_lemmatized.csv"
df.to_csv(fichier, sep=';', index=False, quotechar='"')